# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

/workspace/home


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line)

# creating a smaller event data csv file called event_datafile_full csv \
# that will be used to insert data into the Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Apache Cassandra Setup

### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine (127.0.0.1)

from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1'])
    # To establish connection and begin executing queries, need a session
    session = cluster.connect()
except Exception as e:
    print(e)

### Create Keyspace

In [6]:
# Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS event_data 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)
except Exception as e:
    print(e)

### Set Keyspace

In [7]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('event_data')
except Exception as e:
    print(e)

#### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

In [8]:
# Create table
# The idea is to use session_id as partition key and item_in_session as clustering column
# The combination of sessionId and itemInSession together forms the primary key for this table
create_music_history = "CREATE TABLE IF NOT EXISTS music_history \
                            (artist text, \
                            song text, \
                            length double, \
                            session_id int, \
                            item_in_session int, \
                            PRIMARY KEY (session_id, item_in_session) \
                        )"
try:
    session.execute(create_music_history)
except Exception as e:
    print(e)        

In [9]:
# Insert data into table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        insert_statement = "INSERT INTO music_history (artist, song, length, session_id, item_in_session)"
        insert_statement = insert_statement + " VALUES (%s, %s, %s, %s, %s)"
        try:
            session.execute(insert_statement, (line[0], line[9], float(line[5]), int(line[8]), int(line[3])))
        except Exception as e:
            print(e)

In [10]:
# Get results
query = "SELECT artist, song, length FROM music_history WHERE session_id=338 and item_in_session = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
# convert to pandas for better formatting 
pd.DataFrame(list(rows))

artist                             song    length
0  Faithless  Music Matters (Mark Knight Dub)  495.3073

### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [11]:
# Create table
# The combination of user_id and session_id forms the partition key
# item_in_session is the clustering column (this sorts the data)
# Together all three of them form the primary key for the table
create_session_details = "CREATE TABLE IF NOT EXISTS session_details \
                            (artist text, \
                            song text, \
                            first_name text, \
                            last_name text, \
                            item_in_session int, \
                            session_id int, \
                            user_id int, \
                            PRIMARY KEY ((user_id, session_id), item_in_session) \
                        )"
try:
    session.execute(create_session_details)
except Exception as e:
    print(e)        

In [12]:
# Insert data into table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        insert_statement = "INSERT INTO session_details (artist, song, first_name, last_name, item_in_session, session_id, user_id)"
        insert_statement = insert_statement + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        try:
            session.execute(insert_statement, (line[0], line[9], line[1], line[4], int(line[3]), int(line[8]), int(line[10])))
        except Exception as e:
            print(e)

In [13]:
# Get results
query = "SELECT artist, song, first_name, last_name FROM session_details WHERE user_id=10 and session_id=182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

# convert to pandas for better formatting 
pd.DataFrame(list(rows))

artist                                               song  \
0   Down To The Bone                                 Keep On Keepin' On   
1       Three Drives                                        Greece 2000   
2  Sebastien Tellier                                          Kilometer   
3      Lonnie Gordon  Catch You Baby (Steve Pitron & Max Sanna Radio...   

  first_name last_name  
0     Sylvie      Cruz  
1     Sylvie      Cruz  
2     Sylvie      Cruz  
3     Sylvie      Cruz

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [14]:
# Create table
# The idea is to use song as partition key and user_id as clustering column
# The combination of song and user_id together forms the primary key for this table
create_song_history = "CREATE TABLE IF NOT EXISTS song_history \
                            (user_id int, \
                            first_name text, \
                            last_name text, \
                            song text, \
                            session_id int, \
                            item_in_session int, \
                            PRIMARY KEY (song, user_id) \
                        )"
try:
    session.execute(create_song_history)
except Exception as e:
    print(e)        

In [15]:
# Insert data into table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        insert_statement = "INSERT INTO song_history (user_id, first_name, last_name, song, session_id, item_in_session)"
        insert_statement = insert_statement + " VALUES (%s, %s, %s, %s, %s, %s)"
        try:
            session.execute(insert_statement, (int(line[10]), line[1], line[4], line[9], int(line[8]), int(line[3])))
        except Exception as e:
            print(e)

In [16]:
# Get results
query = "SELECT first_name, last_name FROM song_history WHERE song='All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
# convert to pandas for better formatting 
pd.DataFrame(list(rows))

first_name last_name
0  Jacqueline     Lynch
1       Tegan    Levine
2        Sara   Johnson

## Drop the tables before closing out the sessions

In [17]:
# Drop the tables before closing out the sessions
table_names = ["music_history", "session_details", "song_history"]
try:
    for table_name in table_names:
        session.execute(f"DROP TABLE IF EXISTS {table_name}")
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [18]:
session.shutdown()
cluster.shutdown()